<a href="https://colab.research.google.com/gist/Eeman1113/0db0bd70f82ae10e984d71d7a3dea81c/anime-generator-gpt2-method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/eemanmajumder/the-anime-dataset")

In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 42.0 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 6.5 MB 35.1 MB/s 
     |████████████████████████████████| 596 kB 42.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import transformers

In [3]:
batch_size = 10
model_path = 'gpt2_epoch5.bin'
max_seq_len = 300
epochs = 5
data_path = '/content/eda-data.csv'
tokenizer = transformers.GPT2Tokenizer.from_pretrained('gpt2')
model = transformers.GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [4]:
import numpy as np
import re

def remove_source(text):
    cln_text = text
    if '(Source' in cln_text:
        cln_text,_,_ = cln_text.partition('(Source')
    elif '[Written ' in cln_text:
        cln_text,_,_ = cln_text.partition('[Written')
        
    return cln_text

def clean_synopsis(data):
    # removing hentai and kids tags
    data = data[(data.Hentai != 1) & (data.Kids != 1)]
    synopsis = data.synopsis
    synopsis = synopsis.apply(lambda x: str(x))

    # removing very small synopsis
    synopsis = synopsis.apply(lambda x: x if ((len(str(x).strip().split())<=300) and len(str(x).strip().split())>30  ) else -1)
    synopsis = synopsis[synopsis!=-1]
    
    # removing source text
    synopsis = synopsis.apply(lambda x: remove_source(x))
    
    # removing japanese characters
    synopsis = synopsis.apply(lambda x: re.sub("([^\x00-\x7F])+"," ",x))
    
    # remove symbols
    rx = re.compile('[&#/@`)(;<=\'"$%>]')
    synopsis = synopsis.apply(lambda x: rx.sub('',x))
    synopsis = synopsis.apply(lambda x: x.replace('>',""))
    synopsis = synopsis.apply(lambda x: x.replace('`',""))
    synopsis = synopsis.apply(lambda x: x.replace(')',""))
    synopsis = synopsis.apply(lambda x: x.replace('(',""))
    

    # removing adaptation animes (some relevant might get deleted but there aren`t a lot so we wont be affected as much)
    synopsis = synopsis[synopsis.apply(lambda x: 'adaptation' not in str(x).lower())]    
    synopsis = synopsis[synopsis.apply(lambda x: 'music video' not in str(x).lower())]
    synopsis = synopsis[synopsis.apply(lambda x: 'based on' not in str(x).lower())]
    synopsis = synopsis[synopsis.apply(lambda x: 'spin-off' not in str(x).lower())]
    
    return synopsis.reset_index(drop=True)


class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [5]:
import torch

class AnimeDataset():
    def __init__(self,data):
        self.eos_tok = '<|endoftext|>'
        synopsis = clean_synopsis(data)
        synopsis = synopsis.apply(lambda x: str(x) + self.eos_tok)
        self.synopsis = synopsis.tolist()
        self.pad_tok = tokenizer.encode(['<|pad|>'])
    def __getitem__(self,item):
        synopsis = self.synopsis[item]
        tokens = tokenizer.encode(synopsis)
        mask = [1]*len(tokens)
        
        max_len = max_seq_len
        if max_len>len(tokens):
            padding_len = max_len - len(tokens)
            tokens = tokens + self.pad_tok*padding_len
            mask = mask + [0]*padding_len
        else:
            tokens = tokens[:max_len]
            mask = mask[:max_len]
        
        if tokens[-1]!= tokenizer.encode(self.eos_tok)[0]:
            tokens[-1] = tokenizer.encode(self.eos_tok)[0]
        
        return {'ids':torch.tensor(tokens,dtype = torch.long),
                'mask': torch.tensor(mask,dtype = torch.long),
                'og_synpsis':synopsis}
    
        
        
     
    def __len__(self):
        return len(self.synopsis)



In [6]:
from tqdm import tqdm
import torch
import numpy as np


def train_fn(model,dataloader,optimizer,scheduler,device):
    model.train()
    tk0 = tqdm(dataloader, total = len(dataloader), leave = True, position = 0)
    train_loss = AverageMeter()
    losses = []
    for bi,d in enumerate(tk0):
            
        ids = d['ids'].to(device,dtype = torch.long)
        mask = d['mask'].to(device,dtype = torch.long)
        
        loss,out = model(input_ids = ids, labels = ids, attention_mask  = mask)[:2]
        
        train_loss.update(loss.item())    
        loss.backward()
        losses.append(loss.item())
        optimizer.step()
        scheduler.step()
        model.zero_grad()
        tk0.set_postfix(loss = train_loss.avg)
    return np.mean(losses)        

In [16]:
import pandas as pd
from transformers import AdamW 
from transformers import get_linear_schedule_with_warmup
import torch
def run():
    data = pd.read_csv('content/eda-data.csv')
    dataset = AnimeDataset(data = data)
    dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)
    
    device = 'cuda'
    model.to(device)
    
    optimizer = AdamW(model.parameters(),lr = 0.0001,weight_decay = 0.003)    
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=10,num_training_steps = int(len(data)/batch_size * epochs))
    
    best_loss = 111111
    for epoch in range(epochs):
        loss = train_fn(model,dataloader,optimizer,scheduler,device)
        if loss<best_loss:
            best_loss = loss
            torch.save(model.state_dict(),model_path)
        torch.cuda.empty_cache
    

In [17]:
run()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
100%|██████████| 731/731 [24:25<00:00,  2.00s/it, loss=1.25]


In [24]:
def generate_text(input_text,device = 'cuda',max_len = 300):
  pad_tok = tokenizer.encode(['<|pad|>'])[0]
  model.load_state_dict(torch.load(model_path))
  model.to(device)
  model.eval()

  input_ids = tokenizer.encode(input_text)
  mask = [1]*len(input_ids)

  padding_len = max_seq_len - len(input_ids)
  
  input_ids = input_ids #+ pad_tok*padding_len
  mask = mask + [0]*padding_len

  ids = torch.tensor(input_ids,dtype = torch.long).to(device).unsqueeze(0)
  mask = torch.tensor(mask,dtype = torch.long).to(device).unsqueeze(0)
  
  #print(ids[0])
  sample_out = model.generate(ids, min_length = 30,max_length=max_len, pad_token_id=pad_tok,
                              top_p=0.85, early_stopping=True, do_sample=True, num_beams = 5, no_repeat_ngram_size = 2,num_return_sequences=1)
  
  print('AI Generated Synopsis:\n\n',tokenizer.decode(sample_out[0],skip_special_tokens = True))


In [25]:
generate_text('Manas the chad',device = 'cuda')

AI Generated Synopsis:

 Manas the chad of the Earth is back in action, and this time he has the power to turn the tides of war in the form of a giant robot known as the Dragon. The Dragon is said to be able to save the world from the evils of mankind, but the truth behind its existence is shrouded in mystery. 
